In [4]:
# -*- coding: utf8 -*-

In [56]:
import pandas as pd
import os
import numpy as np

### 加载疾病名称和描述

In [165]:
pd_icd10 = pd.read_excel("datasets/ICD-10.xlsx")
print(pd_icd10.shape)
print(pd_icd10.head(5))
icd10disList = pd_icd10['Disease'].tolist()

(23658, 7)
      Code   Disease SecondCode SecondType FirstCode   FirstType WIKI
1  A00.001   古典生物型霍乱    A00-A09      肠道传染病   A00-B99  某些传染病和寄生虫病  NaN
2  A00.101  埃尔托生物型霍乱    A00-A09      肠道传染病   A00-B99  某些传染病和寄生虫病  NaN
3  A00.901        霍乱    A00-A09      肠道传染病   A00-B99  某些传染病和寄生虫病  NaN
4  A00.902      霍乱轻型    A00-A09      肠道传染病   A00-B99  某些传染病和寄生虫病  NaN
5  A00.903      霍乱中型    A00-A09      肠道传染病   A00-B99  某些传染病和寄生虫病  NaN


In [263]:
def loadFileDisDes(fileName,columnNameDis,columnNameDes ):
    pd_file1 = pd.read_excel(os.path.join(filePath, fileName))
    print(pd_file1.columns)
    disList = pd_file1[columnNameDis].tolist()
    desList = pd_file1[columnNameDes].tolist()
    
    disAliasNewList = list()
    desAliasNewList = list()
    
    for i in range(len(disList)):
        dis = disList[i]
        des = desList[i]
        disArr = dis.replace("、",",").replace("，",",").replace(";",",").split(',')
        #print(dis, disArr)
        for i in range(len(disArr)):
            item = disArr[i]
            if len(item.strip())>0:
                disAliasNewList.append(item)
                desAliasNewList.append(des)
    print(len(disList), "名称拆分后梳理：",len(disAliasNewList))
    return disAliasNewList, desAliasNewList

### 加载三份数据的疾病名称和描述

In [204]:
def loadFiles():
    disListALL = list()
    desListALL = list()
    
    disList1, desList1 = loadFileDisDes('disease汇总全部数据.xlsx','字段1_文本', '症状' )
    disListALL.extend(disList1)
    desList1_ = list()
    for des in desList1:
        des_ = des.replace('介绍分享到','').replace('\n','')
        desList1_.append(des_)
    desListALL.extend(desList1_)
    #名医百科
    disList2, desList2 = loadFileDisDes('baike疾病content.xlsx', 'disease_name', '检查')
    disListALL.extend(disList2)
    desListALL.extend(desList2)
    #39健康网
    disList3, desList3 = loadFileDisDes('jbk39jb.xlsx', '名称', '概述')
    disListALL.extend(disList3)
    desListALL.extend(desList3)
    
    return disListALL, desListALL
disListALL, desListALL = loadFiles()
    

Index(['序号', '字段2', '字段1_文本', '病因', '预防', '并发症', '症状', '检查', '诊断鉴别', '治疗',
       '护理', '饮食保健', '疾病简介', '医保疾病', '患病比例', '易感人群', '传染方式', '并发症.1', '就诊科室',
       '治疗方式', '治疗周期', '治愈率', '常用药品', '治疗费用', '温馨提示'],
      dtype='object')
4714 名称拆分后梳理： 4733
Index(['disease_name', 'disease_url', '检查', '趋势', '病因及病理类型', '常见胎盘异常', '鉴别诊断',
       '补钙方法', '辨证诊断', '母乳不足',
       ...
       '辨证要点', '辩证施治', '副银屑病分型', '术前准备', '遗传要点', '面瘫程度的分级', '常见病因', '支原体感染预防',
       '脊柱骨折分类', '预防精神分裂症的复发'],
      dtype='object', length=373)
6462 名称拆分后梳理： 6480
Index(['名称', '链接', '类别', '概述', '别名', '是否医保', '常用药物', '症状', '病因', '治疗 ', '并发症'], dtype='object')
7795 名称拆分后梳理： 7831


In [205]:
#18971
print("疾病数量：",len(disListALL), len(desListALL))

疾病数量： 19044 19044


In [206]:
def doComputICD(disAliasList):
    num=0
    for item in disAliasList:
        if item in icd10disList:
            num +=1
    print(num, len(disListALL))

    numSet = 0
    numList = list()
    for item in list(set(disAliasList)):
        if item in icd10disList:
            numSet +=1
            numList.append(item)
    print(numSet, len(numList),len(numList)*1.0/23658)

### 统计当前icd10疾病覆盖 

In [207]:
doComputICD(disListALL)

4747 19044
2400 2400 0.1014455997971088


In [163]:
d = {'disease': disListALL, 'des': desListALL}
df = pd.DataFrame(data=d)
df.count()
#df.to_csv("datasets/diseaseAll.res")

des        18510
disease    18971
dtype: int64

### 提取别名，作为疾病实体词典使用

In [250]:
print(len(disListALL),len(desListALL))
disAliasNewList = list()
desAliasNewList = list()
disAliasList_ = list()
#39健康网，提取别名
pd_file1 = pd.read_excel(os.path.join(filePath, 'jbk39jb.xlsx'))
print(pd_file1.columns)
disList = pd_file1['别名'].tolist()
desList = pd_file1['概述'].tolist()
for i in range(len(disList)):
        dis = disList[i]
        des = desList[i]
        disArr = dis.replace("、",",").replace("，",",").replace(";",",").split(',')
        #print(dis, disArr)
        for item in disArr:
            if len(item)>0:
                disAliasList_.append(item)
                disAliasNewList.append(item)
                desAliasNewList.append(des)
print("新别名词典数量：", len(disAliasList_))
print("新别名在ICD10数量：", len(disAliasNewList), len(desAliasNewList))

19044 19044
Index(['名称', '链接', '类别', '概述', '别名', '是否医保', '常用药物', '症状', '病因', '治疗 ', '并发症'], dtype='object')
新别名词典数量： 12439
新别名在ICD10数量： 12439 12439


In [251]:
print(len(disAliasNewList), len(disListALL))
print(len(desAliasNewList), len(desListALL))

12439 19044
12439 19044


### 结果中的疾病和别名汇总,30432

In [252]:
disAliasNewList.extend(disListALL)
print(len(disAliasNewList))


31483


In [253]:
desAliasNewList.extend(desListALL)
print(len(desAliasNewList))

31483


In [254]:
doComputICD(disAliasNewList)

5382 19044
2726 2726 0.11522529376954942


### 最终疾病，在icd10中的，获取疾病名称和描述

In [255]:
diNew = list()
deNew = list()
for i in range(len(disAliasNewList)):
    di = disAliasNewList[i]
    de = desAliasNewList[i]
    
    if di in icd10disList:
        diNew.append(di)
        deNew.append(de)
print(len(diNew), len(deNew))
d = {'symptom': diNew, 'symptomDes': deNew}
df = pd.DataFrame(data=d)
df.count()
df.to_csv("datasets/diseaseMatch.csv")

5382 5382


### 补充疾病名称，从症状的相关疾病中，获取疾病名称

In [256]:
aliasList, desList = loadFileDisDes('jbk39zz.xlsx','名称', '相关疾病' )
mayDisList = list()
for inStr in desList:
    inArr = inStr.replace("、",",").replace("，",",").replace(";",",").replace(" ",',').split(r',')
    for item in inArr:
        if(len(item.strip())>0):
            mayDisList.append(item.replace("...",""))
print(len(mayDisList))

Index(['名称', '链接', '类别', '概述', '病因', '相关疾病'], dtype='object')
6753 名称拆分后梳理： 6834
45142


In [257]:
#考虑相关疾病中，拿症状概述补充
doComputICD(mayDisList)

10997 19044
1673 1673 0.07071603685856792


In [258]:
print(len(set(numList)), len(icd10disList))

2730 23658


In [271]:
disAliasNewList.extend(icd10disList)
disAliasNewList.extend(mayDisList)
print("len source=", len(disAliasNewList))
disAliasList__ = list(set(disAliasNewList))
print("len source=", len(disAliasList__))



len source= 100283
len source= 39850


In [272]:
df = pd.DataFrame(disAliasList__)
df.to_csv("datasets/disease.dic")

### 载入症状数据

In [264]:
def loadSymptomFiles():
    symptomListALL = list()
    symptomdesListALL = list()
    symList1, desList1 = loadFileDisDes('symptom症状全数据.xlsx','症状名称', '介绍' )
    symptomListALL.extend(symList1)
    desList1_ = list()
    for des in desList1:
        des_ = des.replace('介绍分享到','').replace('\n','')
        desList1_.append(des_)
    symptomdesListALL.extend(desList1_)
    
    symList2, desList2 = loadFileDisDes('baike症状content.xls','disease_name', '概述' )
    symptomListALL.extend(symList2)
    symptomdesListALL.extend(desList2)  
    
    symList3, desList3 = loadFileDisDes('jbk39zz.xlsx','名称', '概述' )
    symptomListALL.extend(symList3)
    symptomdesListALL.extend(desList3) 
    
    return symptomListALL, symptomdesListALL
symptomListALL, symptomdesListALL = loadSymptomFiles()

Index(['序号', '字段1', '症状名称', '字段3_链接', '字段3_介绍', '字段3_病因', '字段3_预防', '字段3_检查',
       '字段3_诊断', '介绍', '病因', '预防', '检查', '诊断'],
      dtype='object')
6844 名称拆分后梳理： 6962
Index(['disease_name', 'disease_url', 'category', '概述', '病因及常见疾病', '病因',
       '病因病机', '临床表现', '正常值参考', '检查',
       ...
       '腹痛的感觉', '临床表现.2', '主病', '日常调摄', '治疗方案', '宫缩压力', '病因和病理', '分型',
       'FIGO正常和异常子宫出血相关术语、病因新分类系统', '分期'],
      dtype='object', length=180)
1341 名称拆分后梳理： 1344
Index(['名称', '链接', '类别', '概述', '病因', '相关疾病'], dtype='object')
6753 名称拆分后梳理： 6834


### 症状里面，多个症状的拆分开，规范化写，写入文件

In [265]:
symptomListALL_ = list()
for inStr in symptomListALL:
    inArr = inStr.replace("、",",").replace("，",",").replace(";",",").replace(" ",',').split(r',')
    for item in inArr:
        if(len(item)>0):
            symptomListALL_.append(item.replace("...",""))
print("all=",len(symptomListALL_))
df_ = pd.DataFrame(symptomListALL_)
df_.to_csv("datasets/symptom.dic")


all= 15140


In [266]:
d = {'symptom': symptomListALL, 'symptomDes': symptomdesListALL}
df = pd.DataFrame(data=d)
df.count()
df.to_csv("datasets/symptomAll.res")

In [268]:
num=0
tmpList = list()
tmpList.extend(symptomListALL)
tmpList.extend(disListALL)
for item in tmpList:
    if item in icd10disList:
        num +=1
print(num, len(tmpList))



6330 34184


In [269]:

doComputICD(symptomListALL)

1583 19044
937 937 0.03960605292078789


In [270]:
syNew = list()
desNew = list()
for i in range(len(symptomListALL)):
    sym = symptomListALL[i]
    des = symptomdesListALL[i]
    
    if sym in icd10disList:
        syNew.append(sym)
        desNew.append(des)
print(len(syNew), len(desNew))
d = {'symptom': syNew, 'symptomDes': desNew}
df = pd.DataFrame(data=d)
df.count()
df.to_csv("datasets/symptomMatch.csv")

1583 1583
